In [30]:
import pandas as pd
import numpy as np
import nltk
from gensim.summarization.summarizer import summarize
from gensim.summarization import keywords

survey_data = pd.read_excel('data/raw_data.xlsx')


def clean_data(data):
    # This function takes an array of strings and returns an array of cleaned up strings
    cleaned_data = []
    for row,texts in enumerate(data):
        texts = str(texts)
        texts = texts.lower()
        # remove special characters
        texts = texts.replace(r"(http|@)\S+", "")
        texts = texts.replace(r"::", " ")
        texts = texts.replace(r"’", "")
        texts = texts.replace(r",", " ")
        texts = texts.replace(r"[^a-z\':_]", " ")
        # remove repetition
        #pattern = re.compile(r"(.)\1{2,}", re.DOTALL)
        #texts = texts.replace(pattern, r"\1")
        # Transform short negation form
        texts = texts.replace(r"(can't|cannot)", 'can not')
        texts = texts.replace(r"n't", ' not')
        # Remove stop words
        stopwords = nltk.corpus.stopwords.words('english')
        stopwords.remove('not')
        stopwords.remove('nor')
        stopwords.remove('no')
        cleaned_line = ''
        for word in texts.split(" "):
            if word not in stopwords:
                cleaned_line = cleaned_line + " " + word
        cleaned_data.append(cleaned_line)
    return cleaned_data

columns_with_open_responses = ['Q02', 'Q04', 'Q06', 'Q08', 'Q10', 'Q12', 'Q14', 'Q16', 'Q18']
data = survey_data[columns_with_open_responses]
data = data.reset_index()
data = data[1:]
data = data.drop(columns = ['index'])
all_values = []
for column in data:
    this_column_values = data[column].tolist()
    all_values += this_column_values
one_column_df = pd.DataFrame(all_values)
one_column_df = one_column_df.dropna()
data = one_column_df.reset_index()
data = data.drop(columns = ['index'])
data = np.array(data)
data = clean_data(data)
#current_column = 'Q04'
#current_data = raw_data[current_column][1:]
#current_data = current_data.dropna()



/home/niki/anaconda3/lib/python3.8/site-packages/openpyxl/styles/stylesheet.py:221: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")


In [31]:
def clean_up(data):
    return_data = []
    for line in data:
        new_line = ""
        for word in line.split(" "):
            if "\n" in word:
                new_word = word.replace("\n"," ")
            else:
                new_word = word
            new_line = new_line + " " + new_word
        return_data.append(new_line)
    return return_data

In [32]:
current_data = clean_up(data)
all_data_string = " ".join([i for e in current_data for i in e.split(" ")])

text_summarised = summarize(all_data_string, word_count=200)

text_keywords = keywords(all_data_string,split=True, words=20,lemmatize=True)

In [33]:
import re
alphabets= "([A-Za-z])"
prefixes = "(Mr|St|Mrs|Ms|Dr)[.]"
suffixes = "(Inc|Ltd|Jr|Sr|Co)"
starters = "(Mr|Mrs|Ms|Dr|He\s|She\s|It\s|They\s|Their\s|Our\s|We\s|But\s|However\s|That\s|This\s|Wherever)"
acronyms = "([A-Z][.][A-Z][.](?:[A-Z][.])?)"
websites = "[.](com|net|org|io|gov)"

def split_into_sentences(text):
    text = " " + text + "  "
    text = text.replace("\n"," ")
    text = re.sub(prefixes,"\\1<prd>",text)
    text = re.sub(websites,"<prd>\\1",text)
    if "Ph.D" in text: text = text.replace("Ph.D.","Ph<prd>D<prd>")
    text = re.sub("\s" + alphabets + "[.] "," \\1<prd> ",text)
    text = re.sub(acronyms+" "+starters,"\\1<stop> \\2",text)
    text = re.sub(alphabets + "[.]" + alphabets + "[.]" + alphabets + "[.]","\\1<prd>\\2<prd>\\3<prd>",text)
    text = re.sub(alphabets + "[.]" + alphabets + "[.]","\\1<prd>\\2<prd>",text)
    text = re.sub(" "+suffixes+"[.] "+starters," \\1<stop> \\2",text)
    text = re.sub(" "+suffixes+"[.]"," \\1<prd>",text)
    text = re.sub(" " + alphabets + "[.]"," \\1<prd>",text)
    if "”" in text: text = text.replace(".”","”.")
    if "\"" in text: text = text.replace(".\"","\".")
    if "!" in text: text = text.replace("!\"","\"!")
    if "?" in text: text = text.replace("?\"","\"?")
    if "   " in text: text = text.replace("   ",".")
    if "  " in text: text = text.replace("  ",".")
    text = text.replace(".",".<stop>")
    text = text.replace("?","?<stop>")
    text = text.replace("!","!<stop>")
    text = text.replace("<prd>",".")
    sentences = text.split("<stop>")
    sentences = sentences[:-1]
    sentences = [s.strip() for s in sentences]
    return sentences

In [34]:
text_summarised = split_into_sentences(text_summarised)

In [35]:
text_df = pd.DataFrame(text_summarised,columns=['Text'])
keywords_df = pd.DataFrame(text_keywords,columns=['Keywords'])
text_df.to_csv('data/output/summarised_text.csv')
keywords_df.to_csv('data/output/keywords.csv')